In [1]:
import cirq
from math import pi

In [2]:
# Assigning qubits that will be required by the circuit
MAX_QUBITS = 5 # We need to know this number before hand to compile an equivalent circuit
N_QUBITS = MAX_QUBITS
qubits = [cirq.LineQubit(i) for i in range(N_QUBITS)]

In [3]:
# Creating a sample circuit
circuit = cirq.Circuit()

# Applying Hadamard Gate
circuit.append([cirq.H(qubits[0])])
circuit.append([cirq.H(qubits[1])])
circuit.append([cirq.H(qubits[2])])

# Applying CNOT Gate
for i in range(N_QUBITS - 1):
    circuit.append([cirq.CNOT(qubits[i], qubits[i + 1])], cirq.InsertStrategy.NEW)

# Shifting phase along x axis by pi/2 radians
phase_shift = cirq.rx(pi / 2)
for q in qubits:
    circuit.append([phase_shift(q)], cirq.InsertStrategy.INLINE)

# Adding a measurement gate
# for q in qubits:
#     circuit.append([cirq.measure(q)])

In [4]:
# Printing the circuit
print(circuit)

0: ───H───@───────────Rx(0.5π)──────────────
          │
1: ───H───X───@───────Rx(0.5π)──────────────
              │
2: ───H───────X───@───Rx(0.5π)──────────────
                  │
3: ───────────────X───@──────────Rx(0.5π)───
                      │
4: ───────────────────X──────────Rx(0.5π)───


In [5]:
# Available gates for the new compiled circuit
def compile_rx(qubit, theta):
    rx = cirq.rx(theta)
    return rx(qubit)


def compile_rz(qubit, theta):
    rz = cirq.rz(theta)
    return rz(qubit)


def compile_CZ(control_q, target_q):
    return cirq.CZ(control_q, target_q)


# Gates that were used by the input circuit apart from the ones above
def compile_ry(qubit, theta):
    return compile_rx(qubit, pi / 2), compile_rz(qubit, theta), compile_rx(qubit, -pi / 2)


def compile_I(qubit):
    return compile_rx(qubit, 2 * pi)


def compile_H(qubit):
    return compile_rz(qubit, pi / 2), compile_rx(qubit, pi / 2), compile_rz(qubit, pi / 2)


def compile_X(qubit):
    return compile_rx(qubit, pi)


def compile_Y(qubit):
    return compile_ry(qubit, pi)


def compile_Z(qubit):
    return compile_rz(qubit, pi)


def compile_CNOT(control_q, target_q):
    return compile_H(target_q), compile_CZ(control_q, target_q), compile_H(target_q)

In [6]:
def get_compiled_ops(moment):
    compiled_operations = []
    
    # For each operation in a Moment
    for op in moment.operations:

        # Retrieving which gate operation is being performed
        gate = op.gate

        # Compiling gates accordingly:

        # Single qubit gates
        if gate == cirq.I:
            new_op = compile_I(op.qubits[0])

        elif gate == cirq.H:
            new_op = compile_H(op.qubits[0])

        elif gate == cirq.X:
            new_op = compile_X(op.qubits[0])

        elif gate == cirq.Y:
            new_op = compile_Y(op.qubits[0])

        elif gate == cirq.Z:
            new_op = compile_Z(op.qubits[0])
        
        # Two qubit gates
        elif gate == cirq.CNOT:
            new_op = compile_CNOT(op.qubits[0], op.qubits[1])

        elif gate == cirq.CZ:
            new_op = compile_CZ(op.qubits[0], op.qubits[1])
        
        # Single qubit rotation gates along x, y, z axis on Bloch sphere
        # with rotation by theta radians
        else:

            # Retrieveing the value of theta
            theta = gate.exponent * pi

            if gate == cirq.rx(theta):
                new_op = compile_rx(op.qubits[0], theta)

            elif gate == cirq.ry(theta):
                new_op = compile_ry(op.qubits[0], theta)

            elif gate == cirq.rz(theta):
                new_op = compile_rz(op.qubits[0], theta)

        
        compiled_operations.append(new_op)
        
    return compiled_operations

In [173]:
compiled_circuit = cirq.Circuit()

for moment in circuit:
    compiled_operations = get_compiled_ops(moment)
    compiled_circuit.append(compiled_operations)

compiled_circuit

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                        │
1: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                             │
2: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                                                  │
3: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)────────────────────────────────────
                                                                                                                                                                                       │
4: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───

As it can be seen, the number of moments in the input circuit was 6 but in the output circuit it is 23. This increase in the number of moments can be considered as overhead generated by our compiler. So let's see how we can decrease this number.

In [174]:
print("Number of Moments in the input circuit: ", len(circuit))
print("Number of Moments in the compiled output circuit: ", len(compiled_circuit))

Number of Moments in the input circuit:  6
Number of Moments in the compiled output circuit:  23


In [175]:
# Gate combinations that can be merged to I, X, Z. In reversed order for comparison to a stack
# More quantum circuit identities can be added here

can_be_merged_to_i = [
    [cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(0), cirq.rz(pi), cirq.rx(pi / 2)],         # equivalent to reversed --Y-Y--
    [cirq.rz(pi), cirq.rx(-pi / 2), cirq.rz(pi), cirq.rx(3 * pi / 2)],                 # equivalent to reversed --X-Y-Z--
    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi), cirq.rx(pi / 2), cirq.rz(pi / 2)]  # equivalent to reversed --H-H--
]

can_be_merged_to_x = [
    [cirq.rz(pi / 2), cirq.rx(pi), cirq.rz(pi / 2)]                                    # equivalent to --H-Z-H--
]

can_be_merged_to_z = [
    [cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi / 2), cirq.rx(pi), cirq.rz(pi / 2), cirq.rx(pi / 2), cirq.rz(pi / 2)]      # equivalent to --H-X-H--
]

Optimizer functions written here.

In [178]:

def merge_similar(ops):
    """
    """
    opt_ops = []
    q = ops[0].qubits[0]                                        # Since these operations are being performed on qubit "q"
    i = 0
    while i < (len(ops) - 1):
        if type(ops[i].gate) == type(ops[i + 1].gate):

            theta_1 = ops[i].gate.exponent * pi                 # theta from first rotation gate
            theta_2 = ops[i + 1].gate.exponent * pi             # theta from second rotation gate

            if (theta_1 + theta_2) % (2 * pi) == 0:             # Removing any 2*pi rotations
                i += 2
            
            elif ops[i].gate == cirq.rx(theta_1):
                opt_gate = cirq.rx(theta_1 + theta_2)
                opt_ops.append(opt_gate(q))
                i += 2
            
            else:
                opt_gate = cirq.rz(theta_1 + theta_2)
                opt_ops.append(opt_gate(q))
                i += 2
        else:
            opt_ops.append(ops[i])
            i += 1
    opt_ops.append(ops[-1])

    return opt_ops

def merge_to_one_op(ops):
    """
    
    """
    opt_ops = []
    stack = []
    stack_ops = []

    for operation in ops:
        opt_ops.append(operation)

        while(len(opt_ops) > 0):
            stack_ops.append(opt_ops.pop())
            stack.append(stack_ops[-1].gate)

            if stack in can_be_merged_to_i:
                stack.clear()
                stack_ops.clear()

            elif stack in can_be_merged_to_x:
                stack.clear()
                stack_ops.clear()
                X = cirq.rx(pi)
                stack_ops.append(X(q))
                stack.append(X)

            elif stack in can_be_merged_to_z:
                stack.clear()
                stack_ops.clear()
                Z = cirq.rz(pi)
                stack_ops.append(Z(q))
                stack.append(Z)
        
        stack.clear()
        while(len(stack_ops) > 0):
            opt_ops.append(stack_ops.pop())


    return opt_ops

In [179]:
def optimize(circuit):
    """

    """
    opt_circuit = cirq.Circuit(circuit)
    
    cirq.MergeSingleQubitGates(rewriter=merge_similar).optimize_circuit(opt_circuit)
    cirq.MergeSingleQubitGates(rewriter=merge_to_one_op).optimize_circuit(opt_circuit)
    cirq.DropEmptyMoments().optimize_circuit(opt_circuit)
    
    return opt_circuit

optimized_circuit = optimize(compiled_circuit)

optimized_circuit

0: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                       │
1: ────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                            │
2: ─────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)─────────────────────────────────────────────────────────────────────────
                                                                                                                 │
3: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)─────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───@───Rx(0.5π)────────────────────────────────────
                                                                                                                                                      │
4: ───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)──────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───Rz(0.5π)───Rx(0.5π)───Rz(0.5π)───Rx(0.5π)───

Now, after optimization we can see that there has been an improvement, as follows:

In [180]:
print("Number of Moments in the input circuit: ", len(circuit))
print("Number of Moments in the compiled output circuit: ", len(compiled_circuit))
print("Number of Moments in the optimized circuit: ", len(optimized_circuit))

Number of Moments in the input circuit:  6
Number of Moments in the compiled output circuit:  23
Number of Moments in the optimized circuit:  20


As can be seen, after basic optimizations the number of moments were decreased and the optimized circuit is less involved than before. More optimizations can be introduced by simply creating a function which accepts a list of operations on a qubit and return a list of operations on that qubit.

Keep in mind that not all circuits can be improved, for example if a circuit is already simplified by the optimizer's standards than it will not result in a better circuit.

In [181]:
print("Number of Moments in the optimized circuit: ", len(optimized_circuit))
print("Number of Moments in the double optimized circuit: ", len(optimize(optimized_circuit)))

Number of Moments in the optimized circuit:  20
Number of Moments in the double optimized circuit:  20
